In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import optuna
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import TPESampler

from config.config_types import AppConfig
from utils.logging_utils import ExperimentLogger
from training_routine.trainer import Trainer            
from pipeline.walkforward import WFCVGenerator
from pipeline.preprocessing import preprocess
from hyperparams_search.search_utils import sample_hparams_into_cfg
#from hyperparams_search.torch_estimator import TorchFoldEstimator
#from hyperparams_search.randomsearch import RandomSearch

from utils.gpu_test import gpu_test
from utils.paths import CONFIG_DIR
from utils.custom_formatter import setup_logger






from utils.logging_utils import ExperimentLogger
from training_routine.trainer import Trainer            
from pipeline.walkforward import WFCVGenerator
from utils.gpu_test import gpu_test
from utils.paths import CONFIG_DIR, SP500COPY_PATH, SP500_PATH
from utils.custom_formatter import setup_logger

from models import create_model 



/home/corte911/.conda/envs/torch_h200_py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Pipeline

In [4]:
cfg = AppConfig.from_dict(f"{CONFIG_DIR}/debug.yaml")

preprocess(path=SP500_PATH)


wf = WFCVGenerator(config=cfg.walkforward)

df = wf.df_master


17:32:25 - Preprocessing - DEBUG - percentage of nan returns 0.0034% (preprocessing.py:93)
17:32:25 - WFCVGenerator - DEBUG - WFConfig(train=753d, val=251d, test=251d, lags=20, step=251, max_folds=None) (walkforward.py:29)
17:32:25 - WFCVGenerator - DEBUG - Loading data via preprocess() (walkforward.py:44)
17:32:26 - Preprocessing - DEBUG - percentage of nan returns 0.0034% (preprocessing.py:93)
17:32:26 - WFCVGenerator - DEBUG - Loaded df: shape=(3155303, 3) dtypes={'t': dtype('int16'), 'ret': Float64Dtype(), 'permno': Int64Dtype()} head=
 t       ret  permno
 0 -0.012107   10078
 1 -0.062092   10078
 2  0.001742   10078 (walkforward.py:66)


### Training

In [5]:

# setup logger
console_logger = setup_logger("Experiment", level="INFO")

# --- GPU check (PyTorch) ---
gpu_test()
console_logger.info("GPU check complete.")

# -------- load config --------
cfg = AppConfig.from_dict(f"{CONFIG_DIR}/debug.yaml")
print(cfg)

print(cfg.walkforward)

17:32:40 - Experiment - INFO - GPU check complete. (3647976701.py:6)


AppConfig(model=ModelConfig(name='mlp', hparams={'hidden_sizes': [1024, 1024, 1024, 1024, 1024, 1024], 'dropout_rate': 0, 'activation': ['relu', 'relu', 'relu', 'relu', 'relu', 'relu'], 'output_activation': 'linear', 'learning_rate': 0.001}, search={}), trainer=TrainerConfig(hparams={'epochs': 50, 'batch_size': 512, 'lr': 0.001, 'loss': 'mse', 'metrics': ['mae', 'mse', 'dir_acc'], 'weight_decay': 0, 'val_every': 1, 'search': False}, search={}), walkforward=WFConfig(ratio_train=3, ratio_val=1, ratio_test=1, step=251, lags=20, max_folds=None), experiment=ExperimentConfig(name='MLP_linreg', hyperparams_search=False, monitor='val_loss', mode='min', type='price_prediction', n_trials=20, random_state=1234), data={'target_col': 'y'})
WFConfig(ratio_train=3, ratio_val=1, ratio_test=1, step=251, lags=20, max_folds=None)


In [6]:


# -------- data + components --------
#logger = ExperimentLogger(cfg)


wf = WFCVGenerator(config=cfg.walkforward)



17:32:41 - WFCVGenerator - DEBUG - WFConfig(train=753d, val=251d, test=251d, lags=20, step=251, max_folds=None) (walkforward.py:29)
17:32:41 - WFCVGenerator - DEBUG - Loading data via preprocess() (walkforward.py:44)


17:32:42 - Preprocessing - DEBUG - percentage of nan returns 0.0034% (preprocessing.py:93)
17:32:43 - WFCVGenerator - DEBUG - Loaded df: shape=(3155303, 3) dtypes={'t': dtype('int16'), 'ret': Float64Dtype(), 'permno': Int64Dtype()} head=
 t       ret  permno
 0 -0.012107   10078
 1 -0.062092   10078
 2  0.001742   10078 (walkforward.py:66)


In [7]:

# instantiate trainer (PyTorch)
#trainer = Trainer(cfg, logger)

# model input size: number of lags (columns are constant across folds)
input_shape = cfg.walkforward.lags            # int is fine; build_model handles it
max_folds = cfg.walkforward.max_folds

if cfg.model.name.lower() == "cnn1d":
    input_shape = (1, cfg.walkforward.lags)  # (C, L)
elif cfg.model.name.lower() == "mlp":
    input_shape = (cfg.walkforward.lags,)
else:
    console_logger.warning(f"Model: {cfg.model.name} not recognized!")

def make_input_shape(c):
    if cfg.model.name.lower() == "mlp":
        return (c.walkforward.lags, )
    elif cfg.model.name.lower() == "cnn1d":
        return (1, cfg.walkforward.lags)  # (C, L)
    else:
        console_logger.warning(f"Model: {cfg.model.name} not recognized!")
        raise ValueError



# Get bool for search
hyperparams_search = cfg.experiment.hyperparams_search

if not hyperparams_search:
    console_logger.warning("No hyperparams search for this experiment")
    logger = ExperimentLogger(cfg) 
    trainer = Trainer(cfg, logger)
    logger.begin_trial()

# -------- train per fold --------
for fold, data in enumerate(wf.folds()):
    if max_folds is not None and fold >= max_folds:
        break  # allow running subset of folds
    else:
        # Keep model creation inside the loop to avoid weight leakage across folds
        # Keep model creation inside the loop to avoid weight leakage across folds
        model = create_model(cfg.model, input_shape)       

        if fold == 0:
            console_logger.debug(f"model: {model}")

        trainer.fit_eval_fold(model, data, trial=0, fold=fold)

console_logger.warning("Training completed!")



17:32:52 - Experiment - WARNING - No hyperparams search for this experiment (3305620216.py:30)
17:32:52 - ExperimentLogger - WARNING - Experiment with name 'MLP_linreg' already exists: /orcd/home/002/corte911/code/finNN_code/src/price_prediction/experiments/exp_008_MLP_linreg. If the experiment is of type `search`, 'trial_serch_best' will be overwritten!Creating a new trial under this experiment. (logging_utils.py:64)
17:32:52 - ExperimentLogger - INFO - Experiment directory: /orcd/home/002/corte911/code/finNN_code/src/price_prediction/experiments/exp_008_MLP_linreg (logging_utils.py:92)
17:32:52 - ExperimentLogger - INFO - Trial directory: /orcd/home/002/corte911/code/finNN_code/src/price_prediction/experiments/exp_008_MLP_linreg/trial_20251019_173252 (logging_utils.py:135)
17:32:55 - Trainer - INFO - Fitting fold 00 (trial 00) on torch.Size([364349, 21]) samples, val torch.Size([115317, 21]), test torch.Size([115175, 21]) (trainer.py:180)
E1019 17:32:56.087000 3657465 /orcd/home/002/

TorchRuntimeError: Failed running call_function <built-in function linear>(*(FakeTensor(..., device='cuda:0', size=(512, 21)), Parameter(FakeTensor(..., device='cuda:0', size=(1024, 20), requires_grad=True)), Parameter(FakeTensor(..., device='cuda:0', size=(1024,), requires_grad=True))), **{}):
a and b must have same reduction dim, but got [512, 21] X [20, 1024].

from user code:
   File "/orcd/home/002/corte911/code/finNN_code/src/models/mlp.py", line 82, in forward
    return self.net(x)
  File "/home/corte911/.conda/envs/torch_h200_py312/lib/python3.12/site-packages/torch/nn/modules/container.py", line 250, in forward
    input = module(input)
  File "/home/corte911/.conda/envs/torch_h200_py312/lib/python3.12/site-packages/torch/nn/modules/linear.py", line 125, in forward
    return F.linear(input, self.weight, self.bias)

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dyn

test con giuseppe

In [30]:
cfg = AppConfig.from_dict(f"{CONFIG_DIR}/debug.yaml")
wf = WFCVGenerator(config=cfg.walkforward)

df = wf.df_master
df

TypeError: adjust_for_splits() got an unexpected keyword argument 'base_cols'

In [25]:
df.drop(columns=['window'], inplace=True)
df['y'] = df['y'].apply(lambda x: 1 if x >= 0 else 0 if x < 0 else 0)
train_data = df.iloc[:100000].copy()
test_data = df.iloc[100001:150000].copy()

In [26]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

cfg = AppConfig.from_dict(f"{CONFIG_DIR}/debug.yaml")
wf = WFCVGenerator(config=cfg.walkforward)

train_df = train_data
test_df = test_data

X_train_df = train_df.drop('y', axis=1)
y_train_df = train_df['y']
X_test_df = test_df.drop('y', axis=1)
y_test_df = test_df['y']

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_df)
X_test_scaled = scaler.transform(X_test_df)

# Convert to PyTorch Tensors
X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train_df.values, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test = torch.tensor(y_test_df.values, dtype=torch.float32).view(-1, 1)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)

# --- 3. Define the Neural Network ---
class SimpleNet(nn.Module):
    def __init__(self, input_size):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

# --- 4. Training ---
# Instantiate the model
input_features = X_train.shape[1]
model = SimpleNet(input_size=input_features)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training finished.")

# --- 5. Evaluation ---
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # No need to track gradients for evaluation
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f'Test MSE Loss: {test_loss.item():.4f}')

# You can inspect some predictions
print("\nSample Predictions vs Actuals:")
for i in range(5):
    print(f"Prediction: {predictions[i].item():.2f}, Actual: {y_test[i].item():.2f}")

Epoch [10/50], Loss: 0.7321
Epoch [20/50], Loss: 0.6714
Epoch [30/50], Loss: 0.7076
Epoch [40/50], Loss: 0.7103
Epoch [50/50], Loss: 0.6012
Training finished.
Test MSE Loss: 0.7432

Sample Predictions vs Actuals:
Prediction: 0.24, Actual: 0.00
Prediction: -0.09, Actual: 1.00
Prediction: -0.40, Actual: 1.00
Prediction: -0.65, Actual: 1.00
Prediction: 0.50, Actual: 1.00


In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

gpu_test()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train_df.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test = torch.tensor(y_test_df.values, dtype=torch.float32).view(-1, 1)

X_train = X_train_tensor.unsqueeze(1)
X_test = X_test_tensor.unsqueeze(1)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)

# --- 3. Define the CNN ---
class SimpleCNN(nn.Module):
    def __init__(self, num_features):
        super(SimpleCNN, self).__init__()
        # Input shape: (batch_size, 1, num_features)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        # The size of the flattened layer will be out_channels * num_features
        self.fc1 = nn.Linear(16 * num_features, 32)
        self.output_layer = nn.Linear(32, 1)

    def forward(self, x):
        # x shape: (batch_size, 1, num_features)
        x = self.relu(self.conv1(x))
        # x shape: (batch_size, 16, num_features)
        x = self.flatten(x)
        # x shape: (batch_size, 16 * num_features)
        x = self.relu(self.fc1(x))
        x = self.output_layer(x)
        return x

# --- 4. Training ---
# Instantiate the model
input_features = X_train.shape[2] # Number of features is the last dimension
model = SimpleCNN(num_features=input_features)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training finished.")

# --- 5. Evaluation ---
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f'Test MSE Loss: {test_loss.item():.4f}')

# You can inspect some predictions
print("\nSample Predictions vs Actuals:")
for i in range(5):
    print(f"Prediction: {predictions[i].item():.2f}, Actual: {y_test[i].item():.2f}")

Using device: cuda
Epoch [10/50], Loss: 0.0011
Epoch [20/50], Loss: 0.0004
Epoch [30/50], Loss: 0.0009
Epoch [40/50], Loss: 0.0007
Epoch [50/50], Loss: 0.0019
Training finished.
Test MSE Loss: 0.0015

Sample Predictions vs Actuals:
Prediction: 0.00, Actual: -0.04
Prediction: 0.00, Actual: 0.03
Prediction: 0.00, Actual: 0.02
Prediction: 0.00, Actual: 0.01
Prediction: 0.00, Actual: 0.05
